In [6]:
from typing import List
import gurobipy as gp
from gurobipy import GRB, Model, Var
from pydantic import BaseModel, Field
from decisify.optimizer import OptModel
from decisify.core import OptInputModel, OptOutputModel
from decisify.explainer import GurobiInterrogator

In [7]:
# consider a simple, small transportation network
# Sets
warehouses = ["W1", "W2"]
customers = ["C1", "C2", "C3"]

# Parameters
supply = {"W1": 20, "W2": 30}
demand = {"C1": 10, "C2": 10, "C3": 30}
cost ={"W1": {"C1": 2, "C2": 3, "C3": 1},
         "W2": {"C1": 4, "C2": 1, "C3": 3}}

In [8]:
# Define the input data to the optimizer
class TransportationInputs(OptInputModel):
    """Input data for the transportation problem"""
    warehouses: list[str] = Field(description="List of warehouses")
    customers: list[str] = Field(description="List of customers")
    supply: dict[str, int] = Field(description="Supply at each warehouse")
    demand: dict[str, int] = Field(description="Demand at each customer")
    cost: dict[str, dict[str, int]] = Field(description="Cost of transportation between warehouses and customers")


In [9]:
# Define the output data to the optimizer
class TransportationOutputs(OptOutputModel):
    """Output data for the transportation problem"""
    status: int = Field(description="Status of the optimization model")
    total_cost: float = Field(description="Total transportation cost")
    solution: dict[tuple[str, str], float] = Field(description="Solution for the transportation problem")


In [10]:
class TransportationModel(OptModel):
    """ Optimization model for the transportation problem"""

    def _generate_opt_model(self, input_data: TransportationInputs):
        """ Generate the optimization model for the transportation problem"""
        # Create a new model
        model = gp.Model("transportation")

        # Create variables
        x = model.addVars(input_data.warehouses, input_data.customers, name="x")

        # Set objective
        model.setObjective(gp.quicksum(input_data.cost[w][c] * x[w, c] for w in input_data.warehouses for c in input_data.customers), GRB.MINIMIZE)

        # Add supply constraints
        model.addConstrs((gp.quicksum(x[w, c] for c in input_data.customers) <= input_data.supply[w] for w in input_data.warehouses), "Supply")

        # Add demand constraints
        model.addConstrs((gp.quicksum(x[w, c] for w in input_data.warehouses) == input_data.demand[c] for c in input_data.customers), "Demand")

        self.model = model
        self.decision_vars = {'x': x}


    def get_solution(self, input_data: TransportationInputs) -> TransportationOutputs:
        """ Get the solution for the transportation problem"""
        self._generate_opt_model(input_data)
        if self.is_solved():
            for w in input_data.warehouses:
                for c in input_data.customers:
                    if self.decision_vars['x'][w, c].x > 0:
                        print(f"Send {self.decision_vars['x'][w, c].x} units from {w} to {c} at cost {input_data.cost[w][c]}")

            # Print the total cost
            print(f"Total transportation cost: {self.model.objVal}")

            return TransportationOutputs(status=GRB.OPTIMAL, total_cost=self.model.objVal, solution={self.decision_vars['x'][w, c]: self.decision_vars['x'][w, c].x for w in input_data.warehouses for c in input_data.customers})
        else:
            self.solve_model()
            # Print the total cost
            if self.model.Status == GRB.OPTIMAL or self.model.Status == GRB.INTEGER or self.model.Status == GRB.TIME_LIMIT:
                print(f"Total transportation cost: {self.model.objVal}")
                solution = {
                                (w, c): self.decision_vars['x'][w, c].x
                                for w in input_data.warehouses
                                for c in input_data.customers
                                if self.decision_vars['x'][w, c].x > 1e-6  # Filter near-zero values for clarity
                            }
                return TransportationOutputs(status=self.model.Status, total_cost=self.model.objVal, solution=solution)
            else:
                # run IIS to get infeasible constraints
                # Run IIS to identify infeasible constraints
                self.model.computeIIS()

                # Collect IIS details
                iis_details = "Irreducible Inconsistent Subsystem (IIS) Details:\n"

                # Constraints in the IIS
                for constr in self.model.getConstrs():
                    if constr.IISConstr:
                        iis_details += f"Constraint: {constr.ConstrName}\n"

                # Variables with bound issues in the IIS
                for var in self.model.getVars():
                    if var.IISLB:
                        iis_details += f"Variable {var.VarName} has an issue with its lower bound.\n"
                    if var.IISUB:
                        iis_details += f"Variable {var.VarName} has an issue with its upper bound.\n"

                return f"Model is infeasible. {iis_details}"




In [11]:

# Instantiate the input adata model
input_data = TransportationInputs(
warehouses=warehouses,
customers=customers,
supply=supply,
demand=demand,
cost=cost)

In [12]:
# Create an instance of the transportation model
trnsprt_model = TransportationModel()

# Solve the model and print the solution
solution = trnsprt_model.get_solution(input_data)
print(solution.model_dump_json())

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0xe7227635
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 3e+01]
Presolve removed 5 rows and 6 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.000000000e+02
Total transportation cost: 100.0
{"status":2,"total_cost":100.0,"solution":{"W1,C1":10.0,"W1,C3":10.0,"W2,C2":10.0,"W2,C3":20.

In [13]:
interrogator = GurobiInterrogator(trnsprt_model, input_data)
answer = interrogator.answer("What is the optimal solution for the transportation problem?")
print(answer)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0xe7227635
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 3e+01]
Presolve removed 5 rows and 6 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.000000000e+02
Total transportation cost: 100.0
optimal solution for the transportation problem


In [15]:
answer = interrogator.answer("How many factories and how many distribution centers are there?")
print(answer)

Two factories and two distribution centers


In [16]:
 #Now, lets assume the user wants to change the supply at warehouse W1 to 20
answer = interrogator.what_if("the courier company just doubled the transportation costs, how does this affect the total cost?")
print(answer)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0x1c648fb1
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 3e+01]
Presolve removed 5 rows and 6 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.000000000e+02
Total transportation cost: 200.0
status=2 total_cost=200.0 solution={('W1', 'C1'): 10.0, ('W1', 'C3'): 10.0, ('W2', 'C2'): 10.0, ('W2', 'C3'): 20.0}


In [17]:
answer = interrogator.what_if("The demand at customer C1 has increased by 100 times, how does this affect the total cost?")
print(answer)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0xa747bb0d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+03]
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Infeasible model
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0400000e+03   1.010000e+03   0.000000e+00      0s

IIS computed: 5 constraints and 0 bounds
IIS runtime